## Import Libraries

In [42]:
import glob
import os
from decimal import Decimal


## Set Paths

In [43]:
# It is the path to the directory where all images are stored
image_dir ='TrafficSign_Dataset/Images'

# It is the path to the directory where all text files corresponding to the images are stored
file_dir ='C:\\Users\\Hp\\Downloads\\Data\\Data\\labels' 


In [44]:
print(os.getcwd())
current_dir ='C:\\Users\\Hp\\Downloads\\Data\\Data\\Images' # PATH TO IMAGE DIRECTORY


C:\Users\Hp\Downloads\Data\Data


## Conversion of x_centre, y_centre, height , width 

The original dataset is in accordance to the darknet YOLO format which contains the bounded boxes in this format:  

        x_centre, y_centre, height, width, label

But we need to convert it into Keras YOLO format which is:  

        label  x_min  y_min  x_max  y_max

The equations we created are:

  
        y_max = (1600 * y_centre) + ( 800 * height ) / 2

        y_min = 1600 * x_centre - y_max


        x_max = ( x_centre * 2720) + ( width * 1360) / 2

        x_min = x_centre * 2720 - x_max

In [45]:
#Storing all files
files = {}
dirs = os.listdir(file_dir)

for filename in dirs:
    file = open(os.path.join(file_dir, filename), 'r')
    files[filename] = file.read() 
    file.close()

#This dictionary will contain the bounded boxes in Keras YOLO format
conversion = dict()

for key,value in files.items():
    lines = str.splitlines(value)
    for line in lines:
        sepline = line.split()
        label = sepline[0]
        x_centre = Decimal(sepline[1])
        y_centre = Decimal(sepline[2])
        width = Decimal(sepline[3])
        height = Decimal(sepline[4])
        ymax = int(((1600*y_centre) + (800*height)) / 2)
        ymin = int(1600*y_centre - ymax)
        xmax = int(((x_centre*2720) + (width*1360)) / 2)
        xmin = int(x_centre*2720-xmax)
        # If the key appears first time then create a new key
        if key not in conversion.keys():
            conversion[key] = [str(xmin)+","+str(ymin)+","+str(xmax)+","+str(ymax)+","+str(label)]
        # If the key already exists then append in the existing key
        else:
            conversion[key].append(str(xmin)+","+str(ymin)+","+str(xmax)+","+str(ymax)+","+str(label))


# Storing Bounded boxes along with Path

In the code below, we are performing two tasks:

1. Creating a train and test split for model training and predictions
2. Creating text files to store bounded boxes in each image along with their paths


In [46]:
# Percentage of images to be used for the test set
percentage_test = 10;

# Create train.txt and test.txt
file_train = open('train.txt', 'w')  
file_test = open('test.txt', 'w')
# Populate train.txt and test.txt
counter = 1  
index_test = round(100 / percentage_test) 

for file in glob.iglob(os.path.join(current_dir, '*.jpg')):  
    title, ext = os.path.splitext(os.path.basename(file))
    if counter == index_test:
        counter = 1
        str1 = " "
        file_test.write(image_dir + "/" + title + '.jpg' +" "+ str1.join(conversion[title+'.txt']) + "\n")
    else:
        str1 = " "
        file_train.write(image_dir + "/" + title + '.jpg' +" "+ str1.join(conversion[title+'.txt']) + "\n")
        counter = counter + 1
file_train.close()
file_test.close()
